In [59]:
import requests
import json

def search_recording(artist_name, release_title):
    base_url = "https://musicbrainz.org/ws/2/release/"
    params = {
        "query": f"artist:\"{artist_name}\" AND release:\"{release_title}\"",
        "fmt": "json"
    }
    headers = {"User-Agent": "Mozilla/5.0 (bittah.pupil@proton.me)"} # Replace with your app info

    response = requests.get(base_url, params=params, headers=headers)
    response.raise_for_status() # Raise an exception for bad status codes
    data = response.json()
    #print(data)
    
    if data and data.get('releases'):
        # Return the MBID of the first matching recording
        return data['releases'][0]['id']
    return None

# Example usage:
artist = "Steely Dan"
album = "Aja"
release_mbid = search_recording(artist, album)
print(f"Recording MBID for '{track}' by '{artist}': {release_mbid}")

Recording MBID for 'Aja' by 'Steely Dan': df722d09-1a86-4463-b1c0-9c82339070b7


In [60]:
def get_musicians_for_recording(release_mbid):
    base_url = f"https://musicbrainz.org/ws/2/release/{release_mbid}"
    params = {
        "inc": "artist-credits+labels+recordings+recording-level-rels+work-rels+work-level-rels+artist-rels",  # Include artist relationships
        "fmt": "json"
    }
    headers = {"User-Agent": "Mozilla/5.0 (bittah.pupil@proton.me)"}

    response = requests.get(base_url, params=params, headers=headers)
    response.raise_for_status()
    data = response.json()
    #print(data)

    musicians = []
    #if data and data.get('artist-credit'):
    #    for credit in data['artist-credit']:
    #        if 'artist' in credit and 'name' in credit['artist']:
    #            musicians.append(credit['artist']['name'])
    #return musicians

    dataTest = data['media'][0]['tracks'][0]['recording']['relations'][2]['artist']['name']
    print(dataTest)

#Example usage:
if release_mbid:
    musicians = get_musicians_for_recording(release_mbid)
    print(f"Musicians on '{album}': {musicians}")
else:
    print(f"Recording for '{album}' by '{artist}' not found.")

Larry Carlton
Musicians on 'Aja': None
